In [1]:
import os, sys
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

logger.info("Application started successfully.")


2025-06-16 08:18:17,920 - INFO - Application started successfully.


In [2]:
# load environment variables

storage_account = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
access_key = os.environ["AZURE_STORAGE_ACCOUNT_ACCESS_KEY"]
NESSIE_URI = os.environ['NESSIE_URI']
REF = "etl"
FULL_PATH_TO_WAREHOUSE = os.environ['WAREHOUSE']
AWS_S3_ENDPOINT = os.environ["AWS_S3_ENDPOINT"]
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY  = os.environ['AWS_SECRET_ACCESS_KEY']

In [3]:
# set pyspark configuration
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("Read_csv_from_azure_blob") \
    .setMaster("local[*]") \
    .set("spark.executor.memory", "500m") \
    .set("spark.driver.memory", "500m")

conf.set("spark.sql.execution.verbose", "false")


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

25/06/16 08:18:18 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [6]:
# create namespace gold
namespace = "GOLD"
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {namespace}")
spark.sql(f"SHOW NAMESPACES IN nessie").show()

+---------+
|namespace|
+---------+
|   BRONZE|
|   SILVER|
|     GOLD|
+---------+



In [7]:
# create etl branch
spark.sql(f"USE REFERENCE main IN nessie")
spark.sql(f"DROP BRANCH IF EXISTS {REF}")
spark.sql(f"CREATE BRANCH {REF} IN nessie FROM main")
spark.sql(f"LIST REFERENCES IN nessie").show()


+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch| etl|b63c6cda9f0a8de3b...|
| Branch|main|b63c6cda9f0a8de3b...|
+-------+----+--------------------+



In [8]:
import notebooks.lib.gold_fact_sales_sql as sql
import importlib
importlib.reload(sql)

source_list = [
    {
        "table_name": "INTERNET_SALES_HEADER",
        "sql_create_table": sql.sql_create_gold_internet_sales_header,
        "sql_merge": sql.sql_merge_gold_internet_sales_header,
        "enabled": True
    },
    {
        "table_name": "INTERNET_SALES_DETAIL",
        "sql_create_table": sql.sql_create_gold_internet_sales_detail,
        "sql_merge": sql.sql_merge_gold_internet_sales_detail,
        "enabled": True
    }
]

In [9]:
# change branch to etl
spark.sql(f"USE REFERENCE main IN nessie").show()

for item in [item for item in source_list if item["enabled"]]:
    table_name = item["table_name"].upper()
    # create table
    # logger.info(f'creating table if not exists:{table_name}')
    spark.sql(item["sql_create_table"])

    # merge data into gold
    # logger.info(f'merging into: {table_name}')
    df = spark.sql(item["sql_merge"])
    # logger.info(f'data merged successfully')



+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch|main|b63c6cda9f0a8de3b...|
+-------+----+--------------------+



25/06/16 08:18:24 WARN S3FileIO: Unclosed S3FileIO instance created by:
	org.apache.iceberg.aws.s3.S3FileIO.initialize(S3FileIO.java:444)
	org.apache.iceberg.CatalogUtil.loadFileIO(CatalogUtil.java:402)
	org.apache.iceberg.CatalogUtil.loadFileIO(CatalogUtil.java:349)
	org.apache.iceberg.nessie.NessieCatalog.initialize(NessieCatalog.java:132)
	org.apache.iceberg.CatalogUtil.loadCatalog(CatalogUtil.java:277)
	org.apache.iceberg.CatalogUtil.buildIcebergCatalog(CatalogUtil.java:331)
	org.apache.iceberg.spark.SparkCatalog.buildIcebergCatalog(SparkCatalog.java:153)
	org.apache.iceberg.spark.SparkCatalog.initialize(SparkCatalog.java:752)
	org.apache.spark.sql.execution.datasources.v2.NessieCatalogBridge.setCurrentRefForSpark(NessieCatalogBridge.java:105)
	org.apache.spark.sql.execution.datasources.v2.UseReferenceExec.runInternal(UseReferenceExec.scala:44)
	org.apache.spark.sql.execution.datasources.v2.NessieExec.run(NessieExec.scala:34)
	org.apache.spark.sql.execution.datasources.v2.V2Command

2025-06-16 08:18:52,032 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,034 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,035 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,036 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,036 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,037 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,038 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,039 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,040 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,041 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,042 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,043 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,043 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,044 - INFO - Closing down clientserver connection
2025-06-16 08:18:52,

In [ ]:
spark.sql(f"MERGE BRANCH {REF} INTO main IN nessie").show()
spark.sql(f"DROP BRANCH {REF} IN nessie").show()

In [ ]:
spark.sql("LIST REFERENCES IN nessie").show()